# An Introduction to WISER, Part 3: Neural Networks

In this part of the tutorial, we use the probabilistic labels of the generative model to train a state-of-the-art recurrent neural network. We will use a bi-LSTM-CRF to improve the generalization of the linked HMM model.

## Training a Neural Network

To train your bi-LSTM-CRF network, you first need to define an [AllenNLP configuration file](https://github.com/allenai/allennlp/blob/master/tutorials/getting_started/walk_through_allennlp/configuration.md). These files provide a flexible way of defining neural networks, detailing all the model parameters. For this tutorial, we have already provided a configuration file in ``training_config/tutorial.jsonnet``, which specifies the data path, LSTM encoder configuration, and cuda device, among other parameters. Feel free to take a look at it and edit it for your own experiments.

You may also define other neural networks to learn from the probabilistic output of the generative model. Our implementation uses a bi-LSTM-CRF, but you can build many interesting pipelines using different models as long as they optimize a noise-aware loss function (see ``modules/conditional_random_field``).

To begin training, we need to call AllenNLP's ``train_model_from_file`` method and pass it in the location of the configuration file and the output directory.

In [1]:
from wiser.data.dataset_readers import weak_label   # You need to import weak_label and WiserCrfTagger
from wiser.models import WiserCrfTagger             # since they are used in the training config. file
from allennlp.commands.train import train_model_from_file

train_model_from_file(parameter_filename='/Users/yueyang/Documents/bats/wiser/test/tutorials_test/tutorials_test.jsonnet',
                      serialization_dir='/Users/yueyang/Documents/bats/wiser/test/output/discriminative/link_hmm', 
                      file_friendly_logging=True, force=True)

/anaconda3/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


0it [00:00, ?it/s]


10it [00:00, 11875.15it/s]



0it [00:00, ?it/s]


10it [00:00, 13074.51it/s]


...


  0%|          | 0/1 [00:00<?, ?it/s]


accuracy: 0.1841, accuracy3: 1.0000, precision-overall: 0.0000, recall-overall: 0.0000, f1-measure-overall: 0.0000, loss: 402.2746 ||: 100%|##########| 1/1 [00:01<00:00,  1.38s/it]


  0%|          | 0/1 [00:00<?, ?it/s]


accuracy: 0.00, accuracy3: 1.00, precision-overall: 0.00, recall-overall: 0.00, f1-measure-overall: 0.00, loss: 439.87 ||: 100%|##########| 1/1 [00:01<00:00,  1.58s/it]


WiserCrfTagger(
  (text_field_embedder): BasicTextFieldEmbedder(
    (token_embedder_bert): PretrainedBertEmbedder(
      (bert_model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): BertLayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (output): BertSelfOutput(
          

Once you finish training your discriminative model, you will find it's output scores in the ``serialization_dir``. 

Then, open the ``metrics.json`` file. If you've followed this tutorial closely, the test F1 should be around 70%. That's more than a six point increase with respect to the generative model! 

Recall should have increased the most, suggesting an improvement in the generalization of the pipeline.

## Predicting Tags

To use your WISER model to make predictions, you can call AllenNLP's Predictor class. You will need to pass in the path to the ``model.tar.gz`` file inside your output directory, alongside the ``sentence-tagger`` predictor configuration.


In [2]:
from allennlp.predictors.predictor import Predictor
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter

predictor = Predictor.from_path(archive_path='/Users/yueyang/Documents/bats/wiser/test/output/discriminative/link_hmm/model.tar.gz', 
                                predictor_name='sentence-tagger')

As a small demonstration of our pipeline, we will use SpaCy's word splitter to show individual tag predictions of the discriminative model.

In [3]:
tokenizer = SpacyWordSplitter(language='en_core_web_sm', pos_tags=False)

sentence = 'The movie The Lord of the Rings: The Return of the King (2003) \
            won all 11 awards for which it was nominated, \
            including the Emmy Award for Best Picture'

# Prints all tokens in the sentence, alongside their predicted tags
for match in zip(tokenizer.split_words(sentence), predictor.predict(sentence)['tags']):
    print(match)

(The, 'I-PERF')
(movie, 'I-AWD')
(The, 'I-AWD')
(Lord, 'I-AWD')
(of, 'I-AWD')
(the, 'I-AWD')
(Rings, 'I-AWD')
(:, 'I-AWD')
(The, 'I-AWD')
(Return, 'I-AWD')
(of, 'I-AWD')
(the, 'I-AWD')
(King, 'I-AWD')
((, 'I-AWD')
(2003, 'I-AWD')
(), 'I-AWD')
(won, 'I-AWD')
(all, 'I-AWD')
(11, 'I-AWD')
(awards, 'I-AWD')
(for, 'I-AWD')
(which, 'I-AWD')
(it, 'I-AWD')
(was, 'I-AWD')
(nominated, 'I-AWD')
(,, 'I-AWD')
(including, 'I-AWD')
(the, 'I-AWD')
(Emmy, 'I-AWD')
(Award, 'I-AWD')
(for, 'I-AWD')
(Best, 'I-AWD')
(Picture, 'I-AWD')


Alternatively, you can use the [AllenNLP predict](https://allenai.github.io/allennlp-docs/api/commands/predict/) command to predict entire .json files at once. You will need to use the ``--predictor sentence-tagger`` and ``--include-package wiser`` arguments if you use this command.

Congratulations! You are done with the introductory tutorial!